In [4]:
```python
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools



ModuleNotFoundError: No module named 'tensorflow'

In [12]:
# Data Pipeline
import zipfile
with zipfile.ZipFile('archive.zip', 'r') as myzip:
        myzip.extractall('data_dir') 

 # Update with your dataset path
batch_size = 32
image_size = (128, 128)

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



NameError: name 'transforms' is not defined

In [ ]:
# Basic Neural Network Model
class BasicNN(nn.Module):
    def __init__(self, num_classes):
        super(BasicNN, self).__init__()
        self.fc1 = nn.Linear(3 * 128 * 128, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.softmax(self.fc2(x))
        return x

num_classes = len(dataset.classes)
model = BasicNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [ ]:
# Train the model
num_epochs = 10
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    train_losses.append(running_loss / len(train_loader))
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_losses.append(val_loss / len(val_loader))

# Plot training/test accuracy and loss curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Evaluate the Model
def plot_confusion_matrix(cm, classes):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

cm = confusion_matrix(y_true, y_pred)
plt.subplot(1, 2, 2)
plot_confusion_matrix(cm, dataset.classes)
plt.show()

# Convolutional Neural Network Model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.softmax(self.fc2(x))
        return x

model_cnn = CNN(num_classes)
optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=0.001)



In [ ]:
# Train the CNN model
train_losses_cnn, val_losses_cnn = [], []

for epoch in range(num_epochs):
    model_cnn.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer_cnn.zero_grad()
        outputs = model_cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_cnn.step()
        running_loss += loss.item()
    
    train_losses_cnn.append(running_loss / len(train_loader))
    
    model_cnn.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model_cnn(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_losses_cnn.append(val_loss / len(val_loader))

# Plot training/test accuracy and loss curves for CNN
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses_cnn, label='Train Loss CNN')
plt.plot(val_losses_cnn, label='Validation Loss CNN')
plt.title('Loss Curves CNN')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Evaluate the CNN Model
y_true_cnn, y_pred_cnn = [], []
model_cnn.eval()
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model_cnn(images)
        _, predicted = torch.max(outputs, 1)
        y_true_cnn.extend(labels.numpy())
        y_pred_cnn.extend(predicted.numpy())